## எம்பெடிங்ஸ்

முந்தைய எடுத்துக்காட்டில், `vocab_size` நீளமுள்ள உயர்-பரிமாண bag-of-words வெக்டர்களில் செயல்பட்டோம், மேலும் குறைந்த பரிமாண positional representation வெக்டர்களிலிருந்து sparse one-hot representation-க்கு வெளிப்படையாக மாற்றினோம். இந்த one-hot representation நினைவகத்தை திறமையாக பயன்படுத்தாது, மேலும் ஒவ்வொரு வார்த்தையும் மற்றவற்றிலிருந்து தனித்துவமாக நடத்தப்படுகிறது, அதாவது one-hot encoded வெக்டர்கள் வார்த்தைகளுக்கு இடையேயான semantical ஒற்றுமையை வெளிப்படுத்துவதில்லை.

இந்த அலகில், **News AG** தரவுத்தொகுப்பை மேலும் ஆராய்வோம். முதலில், தரவுகளை ஏற்றவும், முந்தைய நோட்புக்-லிருந்து சில வரையறைகளைப் பெறவும்.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## எம்பெடிங் என்றால் என்ன?

**எம்பெடிங்** என்ற கருத்து என்பது சொற்களை குறைந்த பரிமாணம் கொண்ட அடர்த்தியான வெக்டர்களால் பிரதிநிதித்துவப்படுத்துவது, இது ஒரு சொல்லின் அர்த்தத்தை சில அளவுக்கு பிரதிபலிக்கிறது. பொருள் கொண்ட சொல் எம்பெடிங்களை எப்படி உருவாக்குவது என்பதை பின்னர் விவாதிப்போம், ஆனால் தற்போது எம்பெடிங்களை ஒரு சொல்லின் வெக்டர் பரிமாணத்தை குறைக்கும் ஒரு வழியாகவே நினைத்துக்கொள்ளலாம்.

எம்பெடிங் லேயர் ஒரு சொல்லை உள்ளீடாக எடுத்து, குறிப்பிட்ட `embedding_size` அளவிலான வெளியீடு வெக்டரை உருவாக்கும். ஒரு வகையில், இது `Linear` லேயருக்கு மிகவும் ஒத்ததாக இருக்கும், ஆனால் ஒரு-ஹாட் என்கோடட் வெக்டரை எடுத்துக்கொள்வதற்குப் பதிலாக, இது ஒரு சொல்லின் எண்ணை உள்ளீடாக எடுத்துக்கொள்ளும்.

எங்கள் நெட்வொர்க்கில் முதல் லேயராக எம்பெடிங் லேயரை பயன்படுத்துவதன் மூலம், நாம் bag-of-words மாடலிலிருந்து **embedding bag** மாடலுக்கு மாற முடியும், இதில் முதலில் எங்கள் உரையில் உள்ள ஒவ்வொரு சொல்லையும் தொடர்புடைய எம்பெடிங்காக மாற்றி, பின்னர் அந்த எம்பெடிங்களிலிருந்து `sum`, `average` அல்லது `max` போன்ற ஒரு தொகுப்புக் செயல்பாட்டை கணக்கிடலாம்.

![ஐந்து வரிசை சொற்களுக்கான எம்பெடிங் வகைப்பாட்டாளரை காட்டும் படம்.](../../../../../translated_images/ta/embedding-classifier-example.b77f021a7ee67eee.webp)

எங்கள் வகைப்பாட்டாளர் நரம்பியல் நெட்வொர்க் எம்பெடிங் லேயருடன் தொடங்கும், பின்னர் தொகுப்பு லேயர், மற்றும் அதன் மேல் லினியர் வகைப்பாட்டாளர்:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### மாறுபடும் வரிசை அளவுகளை கையாளுதல்

இந்த கட்டமைப்பின் விளைவாக, நமது நெட்வொர்க்கிற்கு மினிபேட்ச்களை ஒரு குறிப்பிட்ட முறையில் உருவாக்க வேண்டும். முந்தைய யூனிட்டில், bag-of-words பயன்படுத்தியபோது, ஒரு மினிபேட்சில் உள்ள அனைத்து BoW டென்சர்களும், நமது உரை வரிசையின் உண்மையான நீளத்தை பொருட்படுத்தாமல், `vocab_size` என்ற சம அளவைக் கொண்டிருந்தன. ஆனால், நாம்சொற் பதிகைகளை (word embeddings) பயன்படுத்தத் தொடங்கியவுடன், ஒவ்வொரு உரை மாதிரியிலும் மாறுபடும் எண்ணிக்கையிலான சொற்கள் இருக்கும், மேலும் அந்த மாதிரிகளை மினிபேட்ச்களாக இணைக்கும் போது, சில padding ஐ பயன்படுத்த வேண்டியிருக்கும்.

இதை datasource க்கு `collate_fn` செயல்பாட்டை வழங்கும் அதே தொழில்நுட்பத்தைப் பயன்படுத்தி செய்யலாம்:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### எம்பெடிங் வகைப்பாட்டாளரை பயிற்சி செய்வது

இப்போது நாங்கள் சரியான டேட்டா லோடரை வரையறுத்துவிட்டோம், முந்தைய யூனிட்டில் வரையறுக்கப்பட்ட பயிற்சி செயல்பாட்டைப் பயன்படுத்தி மாடலை பயிற்சி செய்யலாம்:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **குறிப்பு**: நேரத்தை மிச்சப்படுத்துவதற்காக, இங்கு நாங்கள் 25k பதிவுகளுக்கு மட்டுமே பயிற்சி அளிக்கிறோம் (ஒரு முழு யுக்திக்கு குறைவாக), ஆனால் நீங்கள் தொடர்ந்து பயிற்சி அளிக்கலாம், பல யுக்திகளுக்கு பயிற்சி அளிக்க ஒரு செயல்பாட்டை எழுதலாம், மேலும் அதிக துல்லியத்தை அடைய கற்றல் விகித அளவுருவில் பரிசோதிக்கலாம். நீங்கள் சுமார் 90% துல்லியத்தை அடைய முடியும்.


### EmbeddingBag லேயர் மற்றும் மாறுபட்ட நீள வரிசை பிரதிநிதித்துவம்

முந்தைய கட்டமைப்பில், மினிபேட்சில் பொருந்துவதற்காக அனைத்து வரிசைகளையும் ஒரே நீளத்திற்கு பதம் செய்ய வேண்டியிருந்தது. மாறுபட்ட நீள வரிசைகளை பிரதிநிதித்துவப்படுத்த இது மிகவும் திறமையான வழி அல்ல - மற்றொரு அணுகுமுறை **offset** வெக்டரைப் பயன்படுத்துவது, இது ஒரு பெரிய வெக்டரில் சேமிக்கப்பட்ட அனைத்து வரிசைகளின் இடைவெளிகளை வைத்திருக்கும்.

![Offset வரிசை பிரதிநிதித்துவத்தை காட்டும் படம்](../../../../../translated_images/ta/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Note**: மேலே உள்ள படத்தில், நாம் எழுத்துக்களின் வரிசையை காட்டுகிறோம், ஆனால் எங்கள் எடுத்துக்காட்டில் நாம் சொற்களின் வரிசைகளுடன் வேலை செய்கிறோம். இருப்பினும், offset வெக்டருடன் வரிசைகளை பிரதிநிதித்துவப்படுத்தும் பொது கொள்கை மாறாமல் இருக்கும்.

Offset பிரதிநிதித்துவத்துடன் வேலை செய்ய, நாம் [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html) லேயரைப் பயன்படுத்துகிறோம். இது `Embedding` போன்றதே, ஆனால் இது உள்ளடக்க வெக்டர் மற்றும் offset வெக்டரை உள்ளீடாக எடுக்கிறது, மேலும் இது சராசரி லேயரையும் கொண்டுள்ளது, இது `mean`, `sum` அல்லது `max` ஆக இருக்கலாம்.

இங்கே `EmbeddingBag` ஐப் பயன்படுத்தும் மாற்றியமைக்கப்பட்ட நெட்வொர்க் உள்ளது:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

பயிற்சிக்காக தரவுத்தொகுப்பை தயாரிக்க, இடச்சலன வேக்டரை தயாரிக்கும் மாற்ற செயல்பாட்டை வழங்க வேண்டும்:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

குறிப்பு, முந்தைய அனைத்து எடுத்துக்காட்டுகளிலும் மாறாக, எங்கள் நெட்வொர்க் இப்போது இரண்டு அளவுருக்களை ஏற்கிறது: தரவுத் திசை மற்றும் ஆஃப்செட் திசை, அவை வெவ்வேறு அளவுகளில் உள்ளன. அதேபோல, எங்கள் தரவுத் தரிப்பான் எங்களுக்கு 2 இற்கு பதிலாக 3 மதிப்புகளை வழங்குகிறது: உரை மற்றும் ஆஃப்செட் திசைகள் இரண்டும் அம்சங்களாக வழங்கப்படுகின்றன. எனவே, அதைப் பராமரிக்க எங்கள் பயிற்சி செயல்பாட்டை சிறிது சரிசெய்ய வேண்டும்:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## செமாண்டிக் எம்பெடிங்ஸ்: Word2Vec

முந்தைய உதாரணத்தில், மாடல் எம்பெடிங் லேயர் வார்த்தைகளை வெக்டார் பிரதிநிதித்துவமாக மாற்ற கற்றுக்கொண்டது, ஆனால் இந்த பிரதிநிதித்துவத்திற்கு அதிக செமாண்டிக் அர்த்தம் இல்லை. ஒரே மாதிரியான வார்த்தைகள் அல்லது சமமான வார்த்தைகள், சில வெக்டார் தூரத்தின் அடிப்படையில் (எ.கா., யூக்லிடியன் தூரம்) ஒன்றுக்கொன்று அருகிலிருக்கும் வெக்டார்களாக மாறும் வகையில், அப்படிப்பட்ட வெக்டார் பிரதிநிதித்துவத்தை கற்றுக்கொள்வது நல்லது.

அதற்காக, நாங்கள் எங்கள் எம்பெடிங் மாடலை ஒரு குறிப்பிட்ட முறையில் பெரிய அளவிலான உரைத் தொகுப்பில் முன்கூட்டியே பயிற்சி செய்ய வேண்டும். செமாண்டிக் எம்பெடிங்ஸ்களை பயிற்சி செய்யும் முதல் முறைகளில் ஒன்று [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) என்று அழைக்கப்படுகிறது. இது வார்த்தைகளின் விநியோக பிரதிநிதித்துவத்தை உருவாக்க பயன்படுத்தப்படும் இரண்டு முக்கிய கட்டமைப்புகளின் அடிப்படையில் உள்ளது:

 - **தொடர்ச்சியான பை-ஆஃப்-வேர்ட்ஸ்** (CBoW) — இந்த கட்டமைப்பில், சுற்றியுள்ள சூழலிலிருந்து ஒரு வார்த்தையை கணிக்க மாடலை பயிற்சி செய்கிறோம். $ (W_{-2},W_{-1},W_0,W_1,W_2)$ என்ற ngram கொடுக்கப்பட்டால், மாடலின் நோக்கம் $(W_{-2},W_{-1},W_1,W_2)$-இல் இருந்து $W_0$-ஐ கணிக்க வேண்டும்.
 - **தொடர்ச்சியான ஸ்கிப்-கிராம்** என்பது CBoW-க்கு எதிர்மறையாகும். மாடல் தற்போதைய வார்த்தையை கணிக்க சுற்றியுள்ள சூழல் வார்த்தைகளின் சாளரத்தைப் பயன்படுத்துகிறது.

CBoW வேகமாக செயல்படுகிறது, ஆனால் ஸ்கிப்-கிராம் மெதுவாக செயல்படுகிறது, ஆனால் அரிதான வார்த்தைகளை பிரதிநிதித்துவப்படுத்த சிறப்பாக செயல்படுகிறது.

![வார்த்தைகளை வெக்டார்களாக மாற்ற CBoW மற்றும் ஸ்கிப்-கிராம் அல்காரிதங்களை காட்டும் படம்.](../../../../../translated_images/ta/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Google News தரவுத்தொகுப்பில் முன்கூட்டியே பயிற்சி செய்யப்பட்ட word2vec எம்பெடிங்குடன் பரிசோதிக்க, **gensim** நூலகத்தை பயன்படுத்தலாம். கீழே 'neural' என்ற வார்த்தைக்கு மிகவும் ஒத்த வார்த்தைகளை காணலாம்.

> **குறிப்பு:** நீங்கள் முதன்முதலில் வார்த்தை வெக்டார்களை உருவாக்கும் போது, அவற்றை பதிவிறக்கம் செய்ய சில நேரம் ஆகலாம்!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


நாம் வார்த்தையிலிருந்து வெக்டர் எம்பெடிங்குகளை கணக்கிடலாம், அவற்றை வகைப்படுத்தல் மாடலைப் பயிற்சியில் பயன்படுத்தலாம் (தெளிவுக்காக வெக்டரின் முதல் 20 கூறுகளை மட்டுமே காட்டுகிறோம்):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

செமாண்டிக்கல் எம்பெடிங்க்ஸ் பற்றிய சிறந்த விஷயம் என்னவென்றால், அர்த்தங்களை மாற்றுவதற்கு வெக்டர் குறியீட்டை மாற்றலாம். உதாரணமாக, *king* மற்றும் *woman* என்ற வார்த்தைகளுக்கு όσοமட்டும் அருகில் இருக்கும், மற்றும் *man* என்ற வார்த்தைக்கு όσοமட்டும் தூரமாக இருக்கும் ஒரு வார்த்தையை கண்டுபிடிக்கக் கேட்கலாம்:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

CBoW மற்றும் Skip-Grams இரண்டும் "predictive" embedding-கள், ஏனெனில் அவை உள்ளூர் சூழல்களை மட்டுமே கணக்கில் எடுத்துக்கொள்கின்றன. Word2Vec உலகளாவிய சூழலின் நன்மைகளை பயன்படுத்துவதில்லை.

**FastText** என்பது Word2Vec-ஐ அடிப்படையாகக் கொண்டு ஒவ்வொரு வார்த்தைக்கும் மற்றும் ஒவ்வொரு வார்த்தையிலும் உள்ள character n-grams-க்கு vector representations-ஐ கற்றுக்கொள்கிறது. இந்த representations-இன் மதிப்புகள் ஒவ்வொரு பயிற்சி படியில் ஒரு vector-ஆக சராசரி எடுக்கப்படும். இது pre-training-க்கு கூடுதல் கணக்கீடுகளை சேர்த்தாலும், sub-word தகவல்களை word embeddings-ல் குறிக்க உதவுகிறது.

மற்றொரு முறை, **GloVe**, co-occurrence matrix-இன் யோசனையை பயன்படுத்தி, co-occurrence matrix-ஐ neural முறைகளை கொண்டு decomposing செய்து, மேலும் வெளிப்படையான மற்றும் non-linear word vectors-ஐ உருவாக்குகிறது.

நீங்கள் embeddings-ஐ FastText மற்றும் GloVe-ஆக மாற்றி உதாரணத்தை முயற்சிக்கலாம், ஏனெனில் gensim பல்வேறு word embedding மாதிரிகளை ஆதரிக்கிறது.


## PyTorch-ல் முன்பே பயிற்சியளிக்கப்பட்ட எம்பெடிங்குகளை பயன்படுத்துதல்

மேலே உள்ள உதாரணத்தை மாற்றி, எம்பெடிங் லேயரில் உள்ள மேட்ரிக்ஸை Word2Vec போன்ற அர்த்தபூர்வமான எம்பெடிங்குகளால் முன்பே நிரப்பலாம். முன்பே பயிற்சியளிக்கப்பட்ட எம்பெடிங் மற்றும் நமது உரை தொகுப்பின் சொற்கள்தொகை பொருந்தாமல் இருக்க வாய்ப்பு உள்ளது என்பதைக் கருத்தில் கொள்ள வேண்டும், எனவே காணாமல் போன சொற்களுக்கான எடை மதிப்புகளை சீரற்ற (random) மதிப்புகளால் தொடங்குவோம்:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


இப்போது நமது மாதிரியை பயிற்சி செய்யலாம். மாதிரியை பயிற்சி செய்யும் நேரம் முந்தைய உதாரணத்தை விட குறிப்பிடத்தக்க அளவில் அதிகமாக இருக்கும் என்பதை கவனிக்கவும், ஏனெனில் பெரிய எம்பெடிங் லேயர் அளவு மற்றும் அதனால் மிக அதிகமான பராமேட்டர்கள் உள்ளன. மேலும், இதனால், ஓவர்ஃபிட்டிங்கை தவிர்க்க விரும்பினால், நமது மாதிரியை மேலும் பல உதாரணங்களில் பயிற்சி செய்ய வேண்டியிருக்கும்.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

எங்கள் நிலைமையில், துல்லியத்தில் பெரிய அளவில் அதிகரிப்பு காணப்படவில்லை, இது மிகவும் வேறுபட்ட சொற்களஞ்சியங்களால் இருக்கக்கூடும்.  
வேறுபட்ட சொற்களஞ்சியங்களின் பிரச்சினையை சமாளிக்க, கீழ்க்காணும் தீர்வுகளில் ஒன்றை பயன்படுத்தலாம்:  
* எங்கள் சொற்களஞ்சியத்தில் word2vec மாதிரியை மீண்டும் பயிற்சி செய்யவும்  
* முன்பே பயிற்சி செய்யப்பட்ட word2vec மாதிரியின் சொற்களஞ்சியத்துடன் எங்கள் தரவுத்தொகுப்பை ஏற்றவும். தரவுத்தொகுப்பை ஏற்றும்போது பயன்படுத்தப்படும் சொற்களஞ்சியம் ஏற்றும்போது குறிப்பிடப்படலாம்.  

இரண்டாவது அணுகுமுறை எளிதாக தோன்றுகிறது, குறிப்பாக PyTorch `torchtext` கட்டமைப்பில் எம்பெடிங்குகளுக்கான உள்ளமைக்கப்பட்ட ஆதரவு உள்ளதால். உதாரணமாக, GloVe அடிப்படையிலான சொற்களஞ்சியத்தை பின்வரும் முறையில் உருவாக்கலாம்:


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


ஏற்றப்பட்ட சொற்களஞ்சியத்தில் அடிப்படை செயல்பாடுகள் பின்வருமாறு உள்ளன:
* `vocab.stoi` அகராதி மூலம் ஒரு சொல் அதன் அகராதி குறியீட்டில் மாற்றப்படுகிறது
* `vocab.itos` இதற்கு எதிராக செயல்படுகிறது - எண்ணை சொலாக மாற்றுகிறது
* `vocab.vectors` என்பது எம்பெடிங் வெக்டர்களின் வரிசையாகும், எனவே ஒரு சொல் `s`-இன் எம்பெடிங்கை பெற `vocab.vectors[vocab.stoi[s]]` பயன்படுத்த வேண்டும்

இங்கே **kind-man+woman = queen** என்ற சமன்பாட்டை விளக்க எம்பெடிங்குகளை மாற்றுவதற்கான உதாரணம் உள்ளது (சில அளவுகளை சரிசெய்ய வேண்டியிருந்தது, இது செயல்படுவதற்காக):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

அந்த எம்பெடிங்குகளைப் பயன்படுத்தி வகைப்பாட்டாளரைப் பயிற்றுவிக்க, முதலில் GloVe சொற்களஞ்சியத்தைப் பயன்படுத்தி எங்கள் தரவுத்தொகுப்பை குறியாக்க வேண்டும்:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

மேலே காணப்பட்டதுபோல், அனைத்து வெக்டர் எம்பெடிங்க்களும் `vocab.vectors` மேட்ரிக்ஸில் சேமிக்கப்படுகின்றன. எம்பெடிங் லேயரின் எடைகளில் அவற்றை எளிதாக ஏற்றுவதற்கு எளிய நகலெடுப்பை பயன்படுத்துவது மிகவும் எளிதாகிறது:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

இப்போது நமது மாதிரியை பயிற்சி செய்து, நாங்கள் சிறந்த முடிவுகளைப் பெறுகிறோமா என்று பார்ப்போம்:


In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

நாம் துல்லியத்தில் குறிப்பிடத்தக்க அதிகரிப்பை காணவில்லை என்பதற்கான காரணங்களில் ஒன்று, எங்கள் தரவுத்தொகுப்பிலிருந்து சில சொற்கள் முன்பே பயிற்சியளிக்கப்பட்ட GloVe சொற்களஞ்சியத்தில் இல்லை என்பதாலேயே அவை அடிப்படையாகப் புறக்கணிக்கப்படுகின்றன. இந்த உண்மையை சமாளிக்க, நாங்கள் எங்கள் சொந்த தரவுத்தொகுப்பில் எம்பெடிங்குகளை பயிற்சி செய்யலாம்.


## சூழலியல் எம்பெடிங்குகள்

Word2Vec போன்ற பாரம்பரிய முன்பதிவுசெய்யப்பட்ட எம்பெடிங் பிரதிநிதிகளின் முக்கிய குறைபாடுகளில் ஒன்று, சொற்களின் அர்த்தத்தை தெளிவுபடுத்தும் சிக்கலாகும். முன்பதிவுசெய்யப்பட்ட எம்பெடிங்குகள் சொற்களின் சில அர்த்தங்களை சூழலில் பிடிக்க முடிந்தாலும், ஒரு சொல்லின் அனைத்து சாத்தியமான அர்த்தங்களும் ஒரே எம்பெடிங்கில் குறியாக்கம் செய்யப்படுகின்றன. இது பின்னர் பயன்படுத்தப்படும் மாதிரிகளில் சிக்கல்களை ஏற்படுத்தலாம், ஏனெனில் 'play' போன்ற பல சொற்களுக்கு அவை பயன்படுத்தப்படும் சூழலின் அடிப்படையில் வெவ்வேறு அர்த்தங்கள் இருக்கும்.

உதாரணமாக, 'play' என்ற சொல் கீழே உள்ள இரண்டு வாக்கியங்களில் முற்றிலும் வேறுபட்ட அர்த்தங்களை கொண்டுள்ளது:
- நான் தியேட்டரில் ஒரு **நாடகம்** பார்த்தேன்.
- ஜான் தனது நண்பர்களுடன் **விளையாட** விரும்புகிறான்.

மேலே உள்ள முன்பதிவுசெய்யப்பட்ட எம்பெடிங்குகள் 'play' என்ற சொல்லின் இந்த இரு அர்த்தங்களையும் ஒரே எம்பெடிங்கில் பிரதிநிதித்துவப்படுத்துகின்றன. இந்த குறைபாட்டை சமாளிக்க, **மொழி மாதிரி** அடிப்படையில் எம்பெடிங்குகளை உருவாக்க வேண்டும், இது ஒரு பெரிய உரை தொகுப்பில் பயிற்சி செய்யப்படுகிறது மற்றும் சொற்களை வெவ்வேறு சூழல்களில் எப்படி இணைக்க முடியும் என்பதை *அறிகிறது*. சூழலியல் எம்பெடிங்குகளை விவாதிப்பது இந்த பாடத்திட்டத்தின் வரம்புக்கு வெளியே உள்ளது, ஆனால் மொழி மாதிரிகளைப் பற்றி பேசும் அடுத்த அலகில் நாம் அவற்றை மீண்டும் பார்க்கப் போகிறோம்.



---

**குறிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையைப் பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கின்றோம், ஆனால் தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறான தகவல்கள் இருக்கக்கூடும் என்பதை தயவுசெய்து கவனத்தில் கொள்ளுங்கள். அதன் தாய்மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பல்ல.
